<a href="https://colab.research.google.com/github/gersonUb/Tp1-MetodosNumericos/blob/main/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo KNN para numpy

## Importamos lo que necesitamos

In [5]:
import numpy as np
from scipy.stats import mode
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score

## Armamos los Datasets

In [6]:
data_test = pd.read_csv('fashion-mnist_test.csv')

In [ ]:
data_train = pd.read_csv('fashion-mnist_train.csv')

## Analisis exploratorio

Notemos que label (etiquetas) es la clase de cada uno de las columnas.
Cada una de estas columnas representa el valor de un píxel en la imagen. Hay un total de 784 columnas de píxeles porque cada imagen es de 28x28 píxeles (28 * 28 = 784).

In [5]:
data_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103.0,87.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0,0,0,0,0,14,53,99,...,0.0,0.0,0.0,0.0,63.0,53.0,31.0,0.0,0.0,0.0
3,2,0,0,0,0,0,0,0,0,0,...,137.0,126.0,140.0,0.0,133.0,224.0,222.0,56.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(data_test.max())

label         9.0
pixel1        2.0
pixel2        2.0
pixel3        3.0
pixel4       44.0
            ...  
pixel780    233.0
pixel781    227.0
pixel782    179.0
pixel783    182.0
pixel784    107.0
Length: 785, dtype: float64


Tengo 9 **CLASES**

Tambien, vease, que los registros (valores) de los píxeles van de 0 a 255, donde 0 representa el color negro y 255 representa el color blanco.

In [8]:
print(data_test.max().max())

255.0


Ahora lo ideal es dividir los datos en:

In [8]:
x_train = data_train.drop('label',axis=1)
x_test = data_test.drop('label',axis=1)
y_train = data_train['label']
y_test = data_test['label']

In [9]:
y_test.head()

0    0
1    1
2    2
3    2
4    3
Name: label, dtype: int64

solo la columna label, que son las etiquetas a predecir

In [11]:
x_test.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,9,8,0,...,103,87,56,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,14,53,99,17,...,0,0,0,0,63,53,31,0,0,0
3,0,0,0,0,0,0,0,0,0,161,...,137,126,140,0,133,224,222,56,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


no tiene la columna label

## Distancia coseno

En el contexto del cálculo de la distancia coseno, los 𝜇 son 0, lo que significa que los datos están centrados. La varianza tampoco se tiene en cuenta directamente en el cálculo de la distancia coseno, ya que este se basa únicamente en la dirección de los vectores y no en su magnitud.

In [2]:
def distancia_coseno(x,y):
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    dot = np.dot(x, y)
    return 1 -(dot / (norm_x * norm_y))

## Funcion Knn

En principio, ya tenemos tal distancia, la vamos a usar para un punto de los datos de entrenamiento (test) comparandolos con todos los puntos de los datos de evaluacion (train), de tal forma, que guardaremos estos resultados en un array

In [ ]:
def clasificador_KNN(k, X_train, X_test, y_train):
    y_predict = []
    for x_test in X_test:
        distancias = []
        # Calcular las distancias entre el punto de prueba y todos los puntos de entrenamiento
        for x_train in X_train:
            distancia = distancia_coseno(x_test, x_train)
            distancias.append(distancia)


Ahora, tengo 60.000 elementos en la lista distancias de la cual quiero ver quienes son los indices que cumplen que son los mas infimos. Entonces,
si distancias fuera [0.3, 0.2, 0.5, 0.1], entonces np.argsort(distancias) devolvería [3, 1, 0, 2] porque 0.1 es la distancia más pequeña (índice 3), seguida por 0.2 (índice 1), 0.3 (índice 0) y 0.5 (índice 2).

In [ ]:
def clasificador_KNN(k, X_train, X_test, y_train):
    y_predict = []
    for x_test in X_test:
        distancias = []
        # Calcular las distancias entre el punto de prueba y todos los puntos de entrenamiento
        for x_train in X_train:
            distancia = distancia_coseno(x_test, x_train)
            distancias.append(distancia)
        # Obtener los índices de los k vecinos más cercanos
        indices_ordenados = np.argsort(distancias)
        k_vecinos_indices = indices_ordenados[:k]


Ahora necesito obtener las etiquetas de los k vecinos más cercanos (en este caso mi hiperparametro k va a estar dado por cualquiera, nosotros quisiera que ese k sea el que mejor performance nos de) y clasificar el x_test

In [ ]:
def clasificador_KNN(k, X_train, X_test, y_train):
    y_predict = []
    for x_test in X_test:
        distancias = []
        # Calcular las distancias entre el punto de prueba y todos los puntos de entrenamiento
        for x_train in X_train:
            distancia = distancia_coseno(x_test, x_train)
            distancias.append(distancia)
        # Obtener los índices de los k vecinos más cercanos
        indices_ordenados = np.argsort(distancias)
        k_vecinos_indices = indices_ordenados[:k]

        # Obtener las etiquetas de los k vecinos más cercanos
        k_vecinos_etiqueta = []
        for i in k_vecinos_indices:
          k_vecinos_etiqueta.append(y_train[i])

        # Determinar la clase mayoritaria entre los vecinos usando numpy
        moda = mode(k_vecinos_etiqueta).mode[0]

        # Agregar la clase predicha a la lista de resultados
        y_predict.append(moda)

    return y_predict

Supongamos que k_vecinos_indices es igual a [3, 7, 10]. Esto significa que los índices de los tres vecinos más cercanos en el conjunto de entrenamiento (**x_train**) son 3, 7 y 10.
Luego, en el bucle for, iteramos sobre estos índices uno por uno. Por ejemplo, en la primera iteración, i sería igual a 3. Entonces, obtenemos la etiqueta correspondiente en el índice 3 de **y_train** y la agregamos a la lista k_vecinos_etiqueta. Hacemos lo mismo para los otros índices en k_vecinos_indices.
Entonces, el resultado seria : k_vecinos_etiqueta = [y_train[3], y_train[7], y_train[10]] que son las clases. *Y finalmente, la clase más común es la que se asigna como la predicción para el punto x_test*

## Cacular performance

In [ ]:
def calcular_precision(y_pred, y_true):
    # Calcular la precisión como el promedio de aciertos
    aciertos = np.sum(np.array(y_pred) == np.array(y_true))
    precision = aciertos / len(y_true)
    return precision